In [ ]:
import cv2
import numpy as np 
from ultralytics import YOLO  # Assuming you are using YOLOv8 from Ultralytics


c:\Python310\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [16]:
model_path = "./yolov8/runs/segment/train2/weights/best.pt"
image_path = "./yolov8/test/1 (12).jpg"

In [17]:
# Load YOLO model
yolo_model = YOLO(model_path)

# Load image
img = cv2.imread(image_path)
original_img = img.copy()

### Predict using YOLO

In [18]:

conf = 0.5  # Confidence threshold
results = yolo_model.predict(img, conf=conf)


0: 640x640 2 left-maxillarys, 109.3ms
Speed: 4.0ms preprocess, 109.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


### Process segmentation masks

In [21]:


# Process segmentation masks
def extract_maxillary_regions(results, image_path):
      """Extract and crop left and right maxillary regions."""
    
      img = cv2.imread(image_path)
      
      maxillary_regions = []
      if results[0].masks:
         masks = results[0].masks.data.cpu().numpy()
         classes = results[0].boxes.cls.cpu().numpy()

         for mask, cls in zip(masks, classes):
               if cls in [0, 1]:  # Assuming 0=left-maxillary, 1=right-maxillary
                  coords = np.where(mask > 0.5)
                  
                  # Ensure the mask has valid coordinates
                  if coords[0].size > 0 and coords[1].size > 0:
                     y_min, y_max = coords[0].min(), coords[0].max()
                     x_min, x_max = coords[1].min(), coords[1].max()

                     cropped = img[y_min:y_max, x_min:x_max]
                     
                     # Skip if cropped region is empty
                     if cropped.size > 0:
                           resized = cv2.resize(cropped, (128, 128))
                           maxillary_regions.append(resized)
                  else:
                     print(f"Invalid mask detected for class {cls}. Skipping.")
      else:
         print(f"No masks detected in {image_path}.")

      return maxillary_regions  # Returns list of cropped regions (left, right)




### Preprocess Dataset

In [29]:
import os

def load_dataset(dataset_path, results):
    """Load dataset and extract maxillary regions."""
    X, y = [], []
    class_mapping = {"mild": 0, "moderate": 1, "severe": 2}

    for class_name in os.listdir(dataset_path):
        class_folder = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_folder):
            continue

        for file_name in os.listdir(class_folder):
            image_path = os.path.join(class_folder, file_name)
            maxillary_regions = extract_maxillary_regions(results, image_path)
            print(image_path, len(maxillary_regions))
            if len(maxillary_regions) == 2:  # Ensure both regions are detected
                left, right = maxillary_regions
                combined = np.concatenate((left, right), axis=1)
                X.append(combined / 255.0)  # Normalize pixel values
                y.append(class_mapping[class_name])

    return np.array(X), np.array(y)

# Dataset path
dataset_path = f"E:\SLIIT\__4th_Year\Research Project - IT4010\Dataset - Waters View Sinus X Ray\sinusities_dataset"
X, y = load_dataset(dataset_path, results)


E:\SLIIT\__4th_Year\Research Project - IT4010\Dataset - Waters View Sinus X Ray\sinusities_dataset\mild\mild_1.JPEG 0
E:\SLIIT\__4th_Year\Research Project - IT4010\Dataset - Waters View Sinus X Ray\sinusities_dataset\mild\mild_10.JPEG 0
E:\SLIIT\__4th_Year\Research Project - IT4010\Dataset - Waters View Sinus X Ray\sinusities_dataset\mild\mild_100.JPEG 0
E:\SLIIT\__4th_Year\Research Project - IT4010\Dataset - Waters View Sinus X Ray\sinusities_dataset\mild\mild_101.JPEG 0
E:\SLIIT\__4th_Year\Research Project - IT4010\Dataset - Waters View Sinus X Ray\sinusities_dataset\mild\mild_102.JPEG 0
E:\SLIIT\__4th_Year\Research Project - IT4010\Dataset - Waters View Sinus X Ray\sinusities_dataset\mild\mild_103.JPEG 0
E:\SLIIT\__4th_Year\Research Project - IT4010\Dataset - Waters View Sinus X Ray\sinusities_dataset\mild\mild_104.JPEG 0
E:\SLIIT\__4th_Year\Research Project - IT4010\Dataset - Waters View Sinus X Ray\sinusities_dataset\mild\mild_105.JPEG 0
E:\SLIIT\__4th_Year\Research Project - IT40

In [26]:
print(f"Number of samples in X: {len(X)}")
print(f"Number of labels in y: {len(y)}")


Number of samples in X: 0
Number of labels in y: 0


### Define Classification Model

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, to_categorical(y, 3), test_size=0.2, random_state=42)

# Build CNN model
def build_cnn(input_shape=(128, 256, 3)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')  # 3 classes
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_cnn()


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

### Train the model

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

# Save the model
model.save("maxillary_cnn_model.h5")


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Predict on new data
def predict_severity(image_path, yolo_model_result, model):
    regions = extract_maxillary_regions(yolo_model_result, image_path)
    if len(regions) == 2:
        combined = np.concatenate((regions[0], regions[1]), axis=1) / 255.0
        combined = np.expand_dims(combined, axis=0)  # Add batch dimension
        prediction = model.predict(combined)
        class_labels = ["Healthy", "Moderate", "Severe"]
        return class_labels[np.argmax(prediction)]
    else:
        return "Maxillary regions not detected."

# Example prediction
example_path = "./yolov8/test/1 (12).jpg"
print(predict_severity(example_path, yolo_model, model))


In [15]:


# Extract and preprocess maxillary regions
maxillary_regions = extract_maxillary_regions(results, original_img)
print(len(maxillary_regions))

if len(maxillary_regions) != 2:
    raise ValueError("Both left and right maxillary regions must be detected.")

# Combine the left and right regions for classification
left_maxillary = maxillary_regions[0]
right_maxillary = maxillary_regions[1]
combined_regions = np.concatenate((left_maxillary, right_maxillary), axis=1)  # Combine side by side

# Normalize the data
combined_regions = combined_regions / 255.0

# Classification model input
X = np.expand_dims(combined_regions, axis=0)  # Add batch dimension


# Build and Load Model
classification_model = build_classification_model()
classification_model.load_weights("../inceptionV3_updated.h5")  # Load pre-trained weights if available

# Predict Severity
predictions = classification_model.predict(X)
class_labels = ["Healthy", "Moderate", "Severe"]
predicted_class = class_labels[np.argmax(predictions)]
print(f"Predicted Severity: {predicted_class}")

2


ValueError: Layer count mismatch when loading weights from file. Model expected 4 layers, found 2 saved layers.